In [0]:
# # For More RAM
# def function(l):
#     l.append([0]*500000000)
#     return l  

# l=[]
# while True:
#   l=function(l)
  

In [0]:
# Mount your google drive where you've saved your assignment folder
import torch
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My Drive/Project/transformers

/content/gdrive/My Drive/Project/transformers


In [0]:
pip install -r requirements.txt

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 50.5MB/s 
     |████████████████████████████████| 860kB 39.9MB/s 
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609184 sha256=319630aee887155cdb63d081b555fbe7ef87a91e7753938599306619f86a50aa
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=c186082f809022c06d4d4286aae1d94630bdcdc108d6c5b50db76c646ca395f8
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built regex sacremoses


In [0]:
!python setup.py build
!python setup.py install

running build
running build_py
running build_scripts
changing mode of build/scripts-3.6/transformers-cli from 600 to 755
running install
running bdist_egg
running egg_info
writing transformers.egg-info/PKG-INFO
writing dependency_links to transformers.egg-info/dependency_links.txt
writing entry points to transformers.egg-info/entry_points.txt
writing requirements to transformers.egg-info/requires.txt
writing top-level names to transformers.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
writing manifest file 'transformers.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/transformers
copying build/lib/transformers/modeling_camembert.py -> build/bdist.linux-x86_64/egg/transformers
copying build/lib/transformers/modeling_auto.py -> build/bdist.linux-x86_64/egg/transformers
copying build/lib/transformers/modeling_gpt2.py -> build/bdist

In [0]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 2.9MB/s 


In [0]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from transformers import BertModel, BertTokenizer
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME
import os

In [0]:
def run_squad(name):
  !python examples/run_squad.py --log_info 0 \
  --model_type bert \
  --model_name_or_path $name \
  --do_eval \
  --do_lower_case \
  --train_file train-v1.1.json \
  --predict_file Oxygen.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir $name

In [0]:
def run_lm(epoch, name):
  !python examples/run_lm_finetuning.py \
  --num_train_epochs=$epoch --output_dir=bert_base_cased_qa_oxygen \
  --overwrite_output_dir --model_type=bert  \
  --model_name_or_path=$name \
  --do_train   --train_data_file=data/Oxygen.txt --mlm

In [0]:
def hybrid():
  import torch
  from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

  print("Fetching qa component")
  qa_name = "bert_base_cased_qa"
  qa_model = BertModel.from_pretrained(qa_name)
  qa_component = list(qa_model.encoder.children())
  print("Fetching o2 component")
  em_file = "bert_base_cased_qa_oxygen"
  em_model = BertModel.from_pretrained(em_file)
  em_component = list(em_model.encoder.children())
  tokenizer = BertTokenizer.from_pretrained(em_file)
  list_=torch.nn.modules.container.ModuleList()
  tempqa=qa_component[0][:8]
  print("Length of tempqa:", len(tempqa))
  hybrid_component = tempqa.extend(em_component[0][8:12])
  print("Length of tempqa final:", len(hybrid_component))
  hybrid = BertModel.from_pretrained(qa_name)
  hybrid.encoder.children = None
  hybrid.encoder.children = hybrid_component
  print("Length of hybrid final:", len(list(hybrid.encoder.children())[0]))
  print("Creating hybrid component")
  output_dir = "bert_base_cased_hybrid"
  hy_name = output_dir
  output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
  output_config_file = os.path.join(output_dir, CONFIG_NAME)
  model_to_save = hybrid.module if hasattr(hybrid, 'module') else hybrid
# If we save using the predefined names, we can load using `from_pretrained`
  output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
  output_config_file = os.path.join(output_dir, CONFIG_NAME)
  torch.save(model_to_save.state_dict(), output_model_file)
  model_to_save.config.to_json_file(output_config_file)
  
  tokenizer.save_vocabulary(output_dir)
  return hybrid, hy_name, qa_model, qa_name, em_model, em_file


In [0]:
def validate_embeddings(models,layers):
  import torch
  from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
  for m in models:
    print("=============== ",m, " =====================")
    tokenizer = BertTokenizer.from_pretrained(m)
    text = " More specifically, the double bond is the result of sequential, low-to-high energy, or Aufbau, filling of orbitals,\
     and the resulting cancellation of contributions from the 2s electrons, after sequential filling of the low σ and σ* orbitals; \
     σ overlap of the two atomic 2p orbitals that lie along the O-O molecular axis and π overlap of two pairs of atomic 2p orbitals \
     perpendicular to the O-O molecular axis,\
     and then cancellation of contributions from the remaining two of the six 2p electrons after their partial filling of the lowest π and π* orbitals."
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # for tup in zip(tokenized_text, indexed_tokens):
      # print('{:<12} {:>6,}'.format(tup[0], tup[1]))
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    model = BertModel.from_pretrained(m)
    model.eval()

    with torch.no_grad():
      encoded_layers, _ = model(tokens_tensor, segments_tensors)
    # for i, token_str in enumerate(tokenized_text):
      # print (i, token_str)
    token_embeddings = torch.stack(encoded_layers, dim=0)

    token_embeddings.size()
    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    token_embeddings.size()
    token_embeddings = token_embeddings.permute(1,0,2)

    token_embeddings.size()
    token_vecs_sum = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

      # `token` is a [12 x 768] tensor

      # Sum the vectors from the last four layers.
      # sum_vec = torch.sum(token[-4:], dim=0)
      
      l_layer=[]
      for l in layers:
        l_layer.append(token[l])
      # sum_vec = token[10]
      sum_vec = torch.sum(torch.stack(l_layer), dim=0)
      # print(sum_vec.shape)
      
      # Use `sum_vec` to represent `token`.
      token_vecs_sum.append(sum_vec)

    # print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))
    # print('First 5 vector values for each instance of "bank".')
    # print('')
    print("Word 1   ", str(token_vecs_sum[63][:5]))
    print("Word 2  ", str(token_vecs_sum[75][:5]))
    # print("river bank   ", str(token_vecs_sum[19][:5]))
    from scipy.spatial.distance import cosine

    # Calculate the cosine similarity between the word bank 
    # in "bank robber" vs "river bank" (different meanings).
    # diff_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])

    # Calculate the cosine similarity between the word bank
    # in "bank robber" vs "bank vault" (same meaning).
    same_bank = 1 - cosine(token_vecs_sum[63], token_vecs_sum[75])
    same_bank_2 = 1 - cosine(token_vecs_sum[84], token_vecs_sum[112])

    print('Similar words:  %.2f' % same_bank)
    print('Similar words:  %.2f' % same_bank_2)


In [0]:
def save_model(name,model,tokenizer_name,is_model=False):
  if is_model:
    bert_base_cased_qa = model
  else:
    bert_base_cased_qa = BertModel.from_pretrained(name)
  output_dir = name
  output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
  output_config_file = os.path.join(output_dir, CONFIG_NAME)
  model_to_save = bert_base_cased_qa.module if hasattr(bert_base_cased_qa, 'module') else bert_base_cased_qa
# If we save using the predefined names, we can load using `from_pretrained`
  output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
  output_config_file = os.path.join(output_dir, CONFIG_NAME)
  torch.save(model_to_save.state_dict(), output_model_file)
  model_to_save.config.to_json_file(output_config_file)
  tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
  tokenizer.save_vocabulary(output_dir)

In [0]:
def get_model(name):
  return  BertModel.from_pretrained(name)

In [0]:
run_lm(name='bert-base-cased', epoch=4)

12/11/2019 12:48:04 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
12/11/2019 12:48:04 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpg2nbsbpb
100% 313/313 [00:00<00:00, 365077.07B/s]
12/11/2019 12:48:04 - INFO - transformers.file_utils -   copying /tmp/tmpg2nbsbpb to cache at /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.d7a3af18ce3a2ab7c0f48f04dc8daff45ed9a3ed333b9e9a79d012a0dedf87a6
12/11/2019 12:48:04 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.d7a3af18ce3a2ab7c0f48f04dc8daff45ed9a3ed333b9e9a79d012a0dedf87a6
12/11/2019 12:48:04 - INFO - transformers.file_utils -   removing temp file /tmp/tmpg2nbsbpb
12/11

In [0]:
run_squad('bert_base_cased_qa_oxygen')

12/11/2019 12:49:03 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
100% 415/415 [00:05<00:00, 79.76it/s]
Evaluating: 100% 52/52 [00:06<00:00,  8.60it/s]
{
  "exact": 0.0,
  "f1": 3.485692400241957,
  "total": 415,
  "HasAns_exact": 0.0,
  "HasAns_f1": 6.002333386308765,
  "HasAns_total": 241,
  "NoAns_exact": 0.0,
  "NoAns_f1": 0.0,
  "NoAns_total": 174
}
12/11/2019 12:49:27 - WARNING - __main__ -   Results: {'exact': 0.0, 'f1': 3.485692400241957, 'total': 415, 'HasAns_exact': 0.0, 'HasAns_f1': 6.002333386308765, 'HasAns_total': 241, 'NoAns_exact': 0.0, 'NoAns_f1': 0.0, 'NoAns_total': 174}


In [0]:
run_squad('bert-base-cased')

12/11/2019 12:49:52 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
100% 313/313 [00:00<00:00, 369703.51B/s]
100% 435779157/435779157 [00:07<00:00, 56928386.69B/s]
100% 415/415 [00:05<00:00, 78.33it/s]
Evaluating: 100% 52/52 [00:06<00:00,  8.66it/s]
{
  "exact": 0.0,
  "f1": 4.608381517505156,
  "total": 415,
  "HasAns_exact": 0.0,
  "HasAns_f1": 7.9355947293138565,
  "HasAns_total": 241,
  "NoAns_exact": 0.0,
  "NoAns_f1": 0.0,
  "NoAns_total": 174
}
12/11/2019 12:50:26 - WARNING - __main__ -   Results: {'exact': 0.0, 'f1': 4.608381517505156, 'total': 415, 'HasAns_exact': 0.0, 'HasAns_f1': 7.9355947293138565, 'HasAns_total': 241, 'NoAns_exact': 0.0, 'NoAns_f1': 0.0, 'NoAns_total': 174}


In [0]:
!python examples/run_squad.py \
  --model_type bert \
  --model_name_or_path bert_base_cased_qa_oxygen \
  --do_train \
  --do_eval \
  --train_file train-v1.1.json \
  --predict_file Oxygen.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir bert_base_cased_qa_oxygen_squad

12/11/2019 12:52:23 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
100% 87599/87599 [14:47<00:00, 98.74it/s]
Epoch:   0% 0/2 [00:00<?, ?it/s]
Iteration:   0% 0/7406 [00:00<?, ?it/s]
Iteration:   0% 1/7406 [00:00<1:07:41,  1.82it/s]
Iteration:   0% 2/7406 [00:01<1:07:34,  1.83it/s]
Iteration:   0% 3/7406 [00:01<1:06:49,  1.85it/s]
Iteration:   0% 4/7406 [00:02<1:06:34,  1.85it/s]
Iteration:   0% 5/7406 [00:02<1:06:11,  1.86it/s]
Iteration:   0% 6/7406 [00:03<1:05:43,  1.88it/s]
Iteration:   0% 7/7406 [00:03<1:05:38,  1.88it/s]
Iteration:   0% 8/7406 [00:04<1:05:27,  1.88it/s]
Iteration:   0% 9/7406 [00:04<1:05:07,  1.89it/s]
Iteration:   0% 10/7406 [00:05<1:05:31,  1.88it/s]
Iteration:   0% 11/7406 [00:05<1:05:14,  1.89it/s]
Iteration:   0% 12/7406 [00:06<1:05:04,  1.89it/s]
Iteration:   0% 13/7406 [00:06<1:04:58,  1.90it/s]
Iteration:   0% 14/7406 [00:07<1:04:57,  1.90it/s]
Iteration:   0% 15/7406 [00:07<1:04:49, 

In [0]:
pwd

'/content/gdrive/My Drive/Project/transformers'

In [0]:
h,hname, q, qname, e, ename = hybrid()

Fetching qa component
Fetching o2 component
Length of tempqa: 8
Length of tempqa final: 12
Length of hybrid final: 12
Creating hybrid component


In [0]:
validate_embeddings([hname,ename,qname],[10])

===============  bert_base_cased_hybrid  =====================
Word 1    tensor([ 1.2751,  0.2195, -1.4968,  0.2204, -1.4422])
Word 2   tensor([ 1.4255, -0.0468, -0.8963,  0.4861, -0.8967])
Similar words:  0.87
Similar words:  0.84
===============  bert_base_cased_qa_oxygen  =====================
Word 1    tensor([ 1.0292,  0.8891, -1.1446,  0.5303,  0.2548])
Word 2   tensor([ 1.0127,  0.5977, -0.5267,  1.0242, -0.9945])
Similar words:  0.68
Similar words:  0.65
===============  bert_base_cased_qa  =====================
Word 1    tensor([ 1.2751,  0.2195, -1.4968,  0.2204, -1.4422])
Word 2   tensor([ 1.4255, -0.0468, -0.8963,  0.4861, -0.8967])
Similar words:  0.87
Similar words:  0.84


In [0]:
tokenizer = BertTokenizer.from_pretrained("bert_base_cased_qa_oxygen")

In [0]:
text = " More specifically, the double bond is the result of sequential, low-to-high energy, or Aufbau, filling of orbitals,\
     and the resulting cancellation of contributions from the 2s electrons, after sequential filling of the low σ and σ* orbitals; \
     σ overlap of the two atomic 2p orbitals that lie along the O-O molecular axis and π overlap of two pairs of atomic 2p orbitals \
     perpendicular to the O-O molecular axis,\
     and then cancellation of contributions from the remaining two of the six 2p electrons after their partial filling of the lowest π and π* orbitals."
marked_text = "[CLS] " + text + " [SEP]"
tokenized_text = tokenizer.tokenize(marked_text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# for tup in zip(tokenized_text, indexed_tokens):
  # print('{:<12} {:>6,}'.format(tup[0], tup[1]))
segments_ids = [1] * len(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [0]:
layers=[10]

In [0]:
with torch.no_grad():
  encoded_layers, _ = em_model(tokens_tensor, segments_tensors)
# for i, token_str in enumerate(tokenized_text):
  # print (i, token_str)
print(len(encoded_layers))
token_embeddings = torch.stack(encoded_layers, dim=0)

token_embeddings.size()
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.
print(len(token_embeddings[0][0]))
# For each token in the sentence...
for token in token_embeddings:

  # `token` is a [12 x 768] tensor

  # Sum the vectors from the last four layers.
  # sum_vec = torch.sum(token[-4:], dim=0)
  
  l_layer=[]
  for l in layers:
    l_layer.append(token[l])
  # sum_vec = token[10]
  sum_vec = torch.sum(torch.stack(l_layer), dim=0)
  # print(sum_vec.shape)
  
  # Use `sum_vec` to represent `token`.
  token_vecs_sum.append(sum_vec)

1


TypeError: ignored

In [0]:
validate_embeddings(['bert_base_cased_qa'],[10])

===============  bert_base_cased_qa  =====================
Word 1    tensor([ 1.2751,  0.2195, -1.4968,  0.2204, -1.4422])
Word 2   tensor([ 1.4255, -0.0468, -0.8963,  0.4861, -0.8967])
Similar words:  0.87
Similar words:  0.84


In [0]:
print("Fetching qa component")
qa_name = "bert_base_cased_qa"
qa_model = BertModel.from_pretrained(qa_name)
qa_component = list(qa_model.encoder.children())
print("Fetching o2 component")
em_file = "bert_base_cased_qa_oxygen"
em_model = BertModel.from_pretrained(em_file)
em_component = list(em_model.encoder.children())
tokenizer = BertTokenizer.from_pretrained(em_file)
# list_=torch.nn.modules.container.ModuleList()
# hybrid_component = qa_component[0][:8].extend(em_component[0][8:12])
# hybrid = BertModel.from_pretrained(qa_name)

Fetching qa component
Fetching o2 component


In [0]:
model = 

In [0]:
model = BertModel.from_pretrained('bert_base_cased_qa')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

with torch.no_grad():
  a,b=model(torch.tensor([[4]]),torch.tensor([[1]]))

model_2 = BertModel.from_pretrained('bert_base_cased_hybrid')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model_2.eval()

with torch.no_grad():
  a2,b2=model_2(torch.tensor([[4]]),torch.tensor([[1]]))

for i in range(12):
  print((a[i][0][0][0]),(a2[i][0][0][0]))

tensor(0.1671) tensor(0.1671)
tensor(0.7417) tensor(0.7417)
tensor(0.5943) tensor(0.5943)
tensor(1.3036) tensor(1.3036)
tensor(1.6537) tensor(1.6537)
tensor(0.1466) tensor(0.1466)
tensor(-0.1452) tensor(-0.1452)
tensor(-0.1369) tensor(-0.1369)
tensor(0.1974) tensor(0.1974)
tensor(-0.2828) tensor(-0.2828)
tensor(-0.1957) tensor(-0.1957)
tensor(0.1077) tensor(0.1077)


In [0]:
print((b[0][0]),(b2[0][0]))

tensor(-0.4781) tensor(-0.4781)


In [0]:
run_squad('bert_base_cased_hybrid')

12/11/2019 08:24:36 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
100% 415/415 [00:05<00:00, 75.25it/s]
Evaluating: 100% 52/52 [00:06<00:00,  8.67it/s]
{
  "exact": 0.0,
  "f1": 3.9920307537656474,
  "total": 415,
  "HasAns_exact": 0.0,
  "HasAns_f1": 6.874243829098521,
  "HasAns_total": 241,
  "NoAns_exact": 0.0,
  "NoAns_f1": 0.0,
  "NoAns_total": 174
}
12/11/2019 08:25:02 - WARNING - __main__ -   Results: {'exact': 0.0, 'f1': 3.9920307537656474, 'total': 415, 'HasAns_exact': 0.0, 'HasAns_f1': 6.874243829098521, 'HasAns_total': 241, 'NoAns_exact': 0.0, 'NoAns_f1': 0.0, 'NoAns_total': 174}


In [0]:
text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."

# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
After         1,258
stealing     11,569
money         1,948
from          1,121
the           1,103
bank          3,085
vault        13,454
,               117
the           1,103
bank          3,085
r               187
##ob         12,809
##ber         3,169
was           1,108
seen          1,562
fishing       5,339
on            1,113
the           1,103
Mississippi   5,201
river         2,186
bank          3,085
.               119
[SEP]           102


In [0]:
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [0]:
# Convert inputs to PyTorch tensors
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert_base_cased_qa')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [0]:
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [0]:
print ("Number of layers:", len(encoded_layers))
layer_i = 0

print ("Number of batches:", len(encoded_layers[layer_i]))
batch_i = 0

print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))

Number of layers: 12
Number of batches: 1
Number of tokens: 24
Number of hidden units: 768


In [0]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [0]:
run_squad('bert_base_cased_qa')

12/11/2019 16:15:18 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
100% 415/415 [00:05<00:00, 76.23it/s]
Evaluating: 100% 52/52 [00:06<00:00,  8.61it/s]
{
  "exact": 46.98795180722892,
  "f1": 49.20690263897576,
  "total": 415,
  "HasAns_exact": 80.91286307053942,
  "HasAns_f1": 84.73387798827777,
  "HasAns_total": 241,
  "NoAns_exact": 0.0,
  "NoAns_f1": 0.0,
  "NoAns_total": 174
}
12/11/2019 16:15:49 - WARNING - __main__ -   Results: {'exact': 46.98795180722892, 'f1': 49.20690263897576, 'total': 415, 'HasAns_exact': 80.91286307053942, 'HasAns_f1': 84.73387798827777, 'HasAns_total': 241, 'NoAns_exact': 0.0, 'NoAns_f1': 0.0, 'NoAns_total': 174}
